In [1]:
import json
from src.data_processing import print_progress, scores_as_list, load_data, load_cmu
import numpy as np
from gensim import models
from src.pun_algorithms import is_Tom_Swifty


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint


import warnings
warnings.filterwarnings('ignore')

In [2]:
# model = models.KeyedVectors.load_word2vec_format("/home/doogy/Data/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [3]:
task1, task2, task3, min_pairs, strings, pun_strings = load_data()

In [5]:
substitutions = scores_as_list("all_trigram_with_pos")

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 99.9% 

In [ ]:
with open("results/tom_swifties.json") as f:
    tom_swifty_annotations = json.load(f)

In [6]:
with open("data/t1-t2-mappings.json") as f:
    mappings = {int(k): int(v) for k, v in json.load(f).items()}

t2_subs = []
for i, (t1, t2) in enumerate(mappings.items()):
    t2_subs.append(list(sorted(substitutions[t2].items(), key=lambda x: x[1][0][1], reverse=True)))

# Coverage, Precision, Recall, Accuracy

In [23]:
tp = 0
guesses = 0
for i in range(len(task2)):
    if tom_swifty_annotations[i]:
        tp += int(task2[i]['target'] == tom_swifty_annotations[i][0][0])
        guesses += 1
    else:
        if len(t2_subs) == 0:
            continue
        try:
            tp += int(task2[i]['target'] == t2_subs[i][0][0].split()[1])
            guesses += 1
        except:
            pass
print("Coverage: ", guesses / len(task2))
print("Precision: ", tp/guesses)
print("Recall: ", tp/len(task2))
print("F1: ", (2 * (tp/guesses) * (tp/len(task2))) / ((tp/guesses) + (tp/len(task2))))

Coverage:  0.988198269079465
Precision:  0.7659235668789809
Recall:  0.7568843430369787
F1:  0.7613771270280965


# Mean Reciprocal Rank

In [35]:
total_rank = 0
for i in range(len(task2)):
    target = task2[i]['target']
    
    if tom_swifty_annotations[i]:
        ranks = [r[0] for r in tom_swifty_annotations[i]]
        if target in ranks:
            total_rank += 1 / (ranks.index(target) + 1)
    else:
        try:
            ranks = [r[0].split()[1] for r in t2_subs[i]]
            if target in ranks:
                total_rank += 1 / (ranks.index(target) + 1)
        except:
            pass   
    
print("Mean Reciprocal Rank: ", total_rank / len(task2))      

Mean Reciprocal Rank:  0.8117099396800422
